In [ ]:
from google.cloud import storage, bigquery
import pandas as pd
import io

In [ ]:
"""
    Variables and parameters for the script.
    Replace the placeholders with your actual values.
"""
bucket_name = 'vibra-dtan-juridico-anp-input'
blob_name = 'grupos_empresariais.xlsx'
bq_dataset = 'rf_ext_anp'
bq_table = 'grupos_empresariais'

In [ ]:
"""
    Download the Excel file from Google Cloud Storage
"""
storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(blob_name)
data = blob.download_as_bytes()


In [ ]:
"""
    Read the Excel file into a DataFrame.

    Rename the columns to match the expected schema.
"""

df = pd.read_excel(io.BytesIO(data))

df = df.rename(columns={
    "Agente Regulado": "ge_nom_agente_regulado",
    "Empresa": "ge_nom_empresa",
    "Cluster": "ge_txt_cluster",
})

In [ ]:
"""
    Load the DataFrame into BigQuery.
"""
bq_client = bigquery.Client()
table_id = f"{bq_client.project}.{bq_dataset}.{bq_table}"
job = bq_client.load_table_from_dataframe(df, table_id)
job.result()
